<a href="https://colab.research.google.com/github/liscolme/EscapeEarth/blob/main/Interns/Elise/ClassAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## mount google drive to access files

from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
## typical imports
%matplotlib inline
!pip install lightkurve==1.9.0 #b/c we want a specific function
import lightkurve as lk
import numpy as np
import pandas as pd

In [ ]:
class ClassAnalysis:
    #imports needed for class
    import lightkurve as lk
    import numpy as np
    import pandas as pd

    def __init__(self,data):
        self.telescope = 'TESS'

        if type(data)==list:
            if len(data)==2:
                self.targetid = data[0]
                self.sector = data[1]
            else:
                print('List should have 2 values: [ticid, sector]')
        else:
            print('Please format data as follows: [ticid,sector]')
# 		return

    def __repr__(self):
#         str_name = str(self.targetid)
        return "TIC: {}".format(self.targetid)


##functions we'll be adding
#plot_cleaned_lc: ticid, sector
    def plot_cleaned_lc(self):
        '''
        DESCRIPTION
        Scatter plot one cleaned ticid from sectors 14 or 15
        INPUTS
        ticid    -(int)TESS TIC ID
        sector   -(int)the region of oberservation
        OUTPUT
        Plots lightcurve of specified TIC ID
        '''
        lc = self.open_lc(stage='clean')
        try:
            plot = lc.scatter()
        except AttributeError:
            plot = "No data or check sector value"
            pass
        return plot
#open_lc: ticid,sector,stage
    def open_lc(self,stage):
        if stage == 'raw':
            return self.open_raw_lc()
        elif stage == 'clean':
            return self.open_clean_lc()
        else:
            return print('stage parameter not understood; valid entries include: "raw" or "clean"')

#open_raw_lc: ticid,sector
    def open_raw_lc(self):
        try:
            filename = glob.glob('/content/gdrive/My Drive/EscapeEarthData/Sector_{}_rawdata/*{}-*-s_lc.fits'.format(self.sector,self.targetid))
            lc_file = lk.open(filename[0])
            lc = lc_file.PDCSAP_FLUX
        except IndexError:
            lc = "None"
            pass
        return lc

#open_cleaned_lc: ticid,sector
    def open_clean_lc(self):
        try:
            lc_file = lk.open('/content/gdrive/My Drive/EscapeEarthData/Sec{}_cleaned/{}/lc.fits'.format(self.sector, self.targetid))
            lc = lc_file.FLUX
            lc.targetid = self.targetid
            return lc
        except FileNotFoundError:
            lc = "None"
            return lc

#plot_raw_data: ticid,sector
    def plot_raw_data(self):
        try:
            filename = glob.glob('/content/gdrive/My Drive/EscapeEarthData/Sector_{}_rawdata/*{}-*-s_lc.fits'.format(self.sector, self.targetid))
            lc_file = lk.open(filename[0])
            lc = lc_file.PDCSAP_FLUX
        except IndexError:
            lc = "None"
            pass
        return lc.plot()

#Start of Class Analysis Functions
class Analysis:
    #imports needed for class
    import lightkurve as lk
    import numpy as np
    import pandas as pd

#Initalizing the class. Setting global attributes.
    def __init__(self,lc):
        self.telescope = 'TESS'
        self.targetid = lc.targetid
        self.sector = lc.sector
        self.time = lc.time
        self.flux = lc.flux
        self.flux_err = lc.flux_err
        return 

    def __repr__(self):
#         str_name = str(self.targetid)
        return "TIC: {}".format(self.targetid)

    def BLS(self, pN=1000, dN=10):
       # from astropy import units as u
       from astropy.timeseries import BoxLeastSquares
       '''
       Purppose
       ------------------
       A Box Least Squares function to print out the compute stats of the periodogram.
       Parameters
       -------------------
       pN                     - the input for N within our periods function 
       dN                     - the input for N within our duration grid function
       Return
       list of stats in the following the order: period, duration, transit-time, power, depth
       ------------------
       Calculate several statistics of a candidate transit.
       '''
       #assigning parameters to variables
       period =  self.periods(pN)
       duration = self.duration_grid(dN)
       flat_lc, flat_time = self.flatten()
       #t = timestamps
       #y = observations
       t = flat_time #lc.time #time
       y = flat_lc.flux #flux
       #dy is the uncertianty
       model = BoxLeastSquares(t, y, dy= flat_lc.flux_err)
       periodogram = model.power(period,duration)
       max_power = np.argmax(periodogram.power)
       #calculates the max stats w/in the transit
       stats = [periodogram.period[max_power],
                            periodogram.duration[max_power],
                            periodogram.transit_time[max_power],
                            max_power,periodogram.depth[max_power]]
       #stats is the one peak, periodogram is the areay
       return stats

    def periods(self, pN=1000):
        period = np.logspace(-0.523, 1.43, pN, endpoint=True)
        return period

    def duration_grid(self, dN=10):
        duration = np.linspace(.01, 0.298, dN)
        return duration

    def flatten(self):
       lc = lk.LightCurve(self.time, self.flux,self.flux_err )
       lc_flat = lc.flatten()
       flat_time = lc_flat.time - lc_flat.time[0]
       return lc_flat, flat_time


In [ ]:
target_id = 7582633 
target_sector = 14
first_star = [target_id,target_sector]

In [ ]:
#initiate a class with our data
first_target = ClassAnalysis(first_star)

#check class representation of target
print(first_target, '\n', type(first_target,))

TIC: 7582633 
 <class '__main__.ClassAnalysis'>


In [ ]:
## test a few class attributes

first_target.targetid, first_target.sector, first_target.telescope

(7582633, 14, 'TESS')

In [ ]:
first_data = first_target.open_lc(stage = 'clean')

In [ ]:
#initialize 2nd Class

first_analysis = Analysis(first_data)
first_analysis

TIC: 7582633

In [ ]:
mystats = first_analysis.BLS()
mystats

[0.2999162518987651, 0.298, 0.157, 0, 0.00039819819922779106]